In [1]:
import os
import numpy as np
from collections import defaultdict
from obspy.core.trace import Stats
from obspy import Trace, UTCDateTime, Stream

from unet_predict_pb import predict

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def read_data(path):
    with open(path, 'r') as file:
        data = [int(i) for i in file.read().splitlines()]
        return data

In [3]:
# 获取目录及文件名
dic = defaultdict(dict)

root_path = "/Users/cui/Downloads/txt/"
dirs = os.listdir(root_path)
dirs.sort()
dirs.remove(".DS_Store")

for dir in dirs:
    sample_file = os.listdir(root_path + dir)
    sample_file.sort()
    dic[dir] = sample_file

In [4]:
time = dirs[0]
data_files = dic[time]
print(time)
print(data_files)

sta_set = set([i[:-22] for i in data_files])
print(sta_set)

20190105080948
['SD.HAY.00.BHE20190105080948.txt', 'SD.HAY.00.BHN20190105080948.txt', 'SD.HAY.00.BHZ20190105080948.txt', 'SD.RSH.00.BHE20190105080948.txt', 'SD.RSH.00.BHN20190105080948.txt', 'SD.RSH.00.BHZ20190105080948.txt']
{'SD.RSH.00', 'SD.HAY.00'}


In [5]:
def initialStats(data_file):
    st_info = data_file.split('.')

    stats = Stats()
    stats.network = st_info[0]
    stats.station = st_info[1]
    stats.location = st_info[2]
    stats.channel = data_file[-21:-18]
    stats.sampling_rate = 100
    stats.npts = 22001
    stats.starttime = UTCDateTime(int(data_file[-18:-14]), int(data_file[-14:-12]),
                                  int(data_file[-12:-10]), int(data_file[-10:-8]),
                                  int(data_file[-8:-6]), int(data_file[-6:-4]))
    return stats

In [6]:
# 将当前时间戳目录下各台站三份量波形文件格式化为stream，并放入streams中
streams = []
for station in sta_set:
    stream = Stream()

    for data_file in data_files:
        if data_file.find(station):
            stats = initialStats(data_file)
            data = np.array(read_data(root_path + time + "/" + data_file))

            trace = Trace(data=data, header=stats)
            stream.append(trace)

    streams.append(stream)

for st in streams:
    print(st)

3 Trace(s) in Stream:
SD.HAY.00.BHE | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples
SD.HAY.00.BHN | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples
SD.HAY.00.BHZ | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples
3 Trace(s) in Stream:
SD.RSH.00.BHE | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples
SD.RSH.00.BHN | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples
SD.RSH.00.BHZ | 2019-01-05T08:09:48.000000Z - 2019-01-05T08:13:28.000000Z | 100.0 Hz, 22001 samples


In [7]:
window_size = 30
step = 10

start_time = UTCDateTime(streams[0][0].stats.starttime)
end_time = UTCDateTime(streams[0][0].stats.endtime)
print(start_time)
print(end_time)

while True:
    if start_time == end_time:
        break

    if UTCDateTime(start_time + window_size) > end_time:
        start_time = UTCDateTime(end_time - window_size)

    print("Start time:", start_time)
    find_p = False
    for st in streams:
        win_st = st.slice(start_time, start_time + window_size)
        win_st.normalize()
        win_st = win_st.detrend('constant')
        win_st = win_st.filter('bandpass', freqmin=0.5, freqmax=20)

        tpstamp = predict(win_st)
        if tpstamp:
            find_p = True
            print("P-wave timestamp:", UTCDateTime(tpstamp))

    start_time = UTCDateTime(start_time + window_size) if find_p else UTCDateTime(start_time + step)


2019-01-05T08:09:48.000000Z
2019-01-05T08:13:28.000000Z
Start time: 2019-01-05T08:09:48.000000Z
Start time: 2019-01-05T08:09:58.000000Z
Start time: 2019-01-05T08:10:08.000000Z
P-wave timestamp: 2019-01-05T08:10:12.545000Z
Start time: 2019-01-05T08:10:38.000000Z
Start time: 2019-01-05T08:10:48.000000Z
P-wave timestamp: 2019-01-05T08:10:52.615000Z
Start time: 2019-01-05T08:11:18.000000Z
Start time: 2019-01-05T08:11:28.000000Z
Start time: 2019-01-05T08:11:38.000000Z
P-wave timestamp: 2019-01-05T08:11:39.835000Z
Start time: 2019-01-05T08:12:08.000000Z
P-wave timestamp: 2019-01-05T08:12:13.295000Z
Start time: 2019-01-05T08:12:38.000000Z
P-wave timestamp: 2019-01-05T08:12:41.795000Z
Start time: 2019-01-05T08:12:58.000000Z
P-wave timestamp: 2019-01-05T08:13:03.370000Z
